In [ ]:
import torch
import torch.nn as nn
from torchvision.models import vgg16

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = vgg16(weights='VGG16_Weights.DEFAULT')
model.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)   # 기존에 있는모델의 뒷부분만 수정하여 사용
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
import torch
from torchvision.datasets.cifar import CIFAR10
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_dataset = CIFAR10(root = './', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root = './', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from torch.optim import Adam
from tqdm import tqdm
lr = 1e-6
optim = Adam(model.parameters(), lr=lr)
epochs = 1
# 학습루프
for epoch in range(epochs):
    tqdm_obj = tqdm(train_loader,desc=f'epoch : {epoch+1}/{epochs}')
    for data, label in tqdm_obj:    # train_loader에서 배치(batch) 단위로 데이터를 하나씩 꺼내오는 부분
        optim.zero_grad()           # 그래디언트 초기화
        preds = model(data.to(device))      # 순전파 -> 입력 데이터(data)를 모델(model)에 통과시켜 예측값(preds)을 얻는 과정/ .to(device)는 데이터와 모델을 동일한 장치(CPU 또는 GPU)로 보내 연산을 준비하는 코드
        loss = nn.CrossEntropyLoss()(preds, label.to(device))   # 손실(Loss) 계산. 모델의 예측값(preds)과 실제 정답(label)이 얼마나 다른지를 계산
        loss.backward()     # 역전파 -> 계산된 손실(loss)에 대한 각 가중치의 그래디언트(기울기)를 계산
        optim.step()        # 가중치 업데이트

        tqdm_obj.set_postfix(loss=loss.item())
        

torch.save(model.state_dict(), 'vgg16_custiom.pth')    

epoch : 1/1:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch : 1/1:   0%|          | 2/1563 [01:03<13:46:47, 31.78s/it, loss=2.48]


KeyboardInterrupt: 